Чтобы использовать датасет, подготовленный для YOLO, для обучения keras-ocr, вам нужно преобразовать данные в формат, который понимает keras-ocr.

1. Разница между форматами YOLO и Keras-OCR
YOLO формат обычно хранит аннотации в .txt-файлах с координатами bounding box в формате:

Copy
class_id x_center y_center width height
(нормализованные относительно ширины и высоты изображения).

Keras-OCR ожидает данные в виде списка пар (image, [текст1, текст2, ...]), где bounding boxes могут быть в формате [[x1, y1], [x2, y2], [x3, y3], [x4, y4]] (координаты углов полигона).

2. Преобразование YOLO → Keras-OCR

# Шаг 1. Загрузка и парсинг YOLO-аннотаций

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import keras_ocr
from tensorflow.keras.optimizers import Adam


In [3]:
def parse_yolo_annotation(img_path, label_path):
    # Загружаем изображение, чтобы узнать его размеры
    img = cv2.imread(img_path)
    img_h, img_w = img.shape[:2]
    
    # Читаем аннотации из .txt-файла
    with open(label_path, 'r') as f:
        lines = f.readlines()
    
    boxes = []
    for line in lines:
        class_id, x_center, y_center, w, h = map(float, line.strip().split())
        
        # Конвертация из нормализованных координат YOLO в абсолютные
        x_center *= img_w
        y_center *= img_h
        w *= img_w
        h *= img_h
        
        # Вычисление координат углов прямоугольника (x1, y1, x2, y2)
        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)
        
        # Добавляем в формате полигона (4 угла)
        box = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
        boxes.append(box)
    
    return img, boxes

# Шаг 2. Создание датасета для Keras-OCR

Keras-OCR ожидает данные в виде списка пар (изображение, список текстов). Если ваш датасет содержит только цифры, можно сгенерировать метки автоматически (предполагая, что class_id соответствует цифре):

In [4]:
def load_yolo_dataset(data_dir):
    dataset = []
    for img_file in os.listdir(os.path.join(data_dir, "images")):
        if not img_file.endswith(('.jpg', '.png')):
            continue
        
        img_path = os.path.join(data_dir, "images", img_file)
        label_path = os.path.join(data_dir, "labels", img_file.replace('.jpg', '.txt').replace('.png', '.txt'))
        
        if not os.path.exists(label_path):
            continue
        
        img, boxes = parse_yolo_annotation(img_path, label_path)
        
        # Предполагаем, что class_id = цифра (0 → "0", 1 → "1" и т. д.)
        texts = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                class_id = int(line.strip().split()[0])
                texts.append(str(class_id))
        
        dataset.append((img, texts))  # Keras-OCR ожидает (image, [text1, text2, ...])
    
    return dataset

# Шаг 3. Обучение Keras-OCR

In [5]:
# Загрузка датасета
dataset = load_yolo_dataset("/home/lastinm/PROJECTS/DATA/syntetic digits")

train_dataset = dataset[:int(0.8 * len(dataset))]
val_dataset = dataset[int(0.8 * len(dataset)):]

In [12]:
def yolo_to_keras_generator(yolo_dataset, batch_size=8, target_size=(640, 640)):
    """Генератор с фиксированным количеством аннотаций на изображение"""
    while True:
        indices = np.arange(len(yolo_dataset))
        np.random.shuffle(indices)
        
        for start_idx in range(0, len(yolo_dataset), batch_size):
            batch_indices = indices[start_idx:start_idx + batch_size]
            batch_images = []
            batch_texts = []
            batch_boxes = []
            
            for idx in batch_indices:
                img, annotations = yolo_dataset[idx]
                
                # Преобразование изображения
                if len(img.shape) == 2:
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                else:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                img = cv2.resize(img, target_size)
                img = img.astype(np.float32) / 255.0
                
                # Обработка аннотаций
                texts = []
                boxes = []
                
                if isinstance(annotations, str):
                    texts.append(annotations)
                    boxes.append([[0, 0], [target_size[0], 0], 
                                [target_size[0], target_size[1]], [0, target_size[1]]])
                elif isinstance(annotations, list):
                    for ann in annotations[:5]:  # Берем не более 5 аннотаций на изображение
                        if isinstance(ann, str):
                            texts.append(ann)
                            boxes.append([[0, 0], [target_size[0], 0], 
                                         [target_size[0], target_size[1]], [0, target_size[1]]])
                        elif len(ann) >= 2:
                            texts.append(str(ann[0]))
                            scale_x = target_size[0] / img.shape[1]
                            scale_y = target_size[1] / img.shape[0]
                            scaled_box = []
                            for point in ann[1]:
                                scaled_box.append([point[0]*scale_x, point[1]*scale_y])
                            boxes.append(scaled_box)
                
                # Дополняем до 5 аннотаций (если меньше)
                while len(texts) < 5:
                    texts.append("")  # Пустая строка как padding
                    boxes.append([[0, 0], [0, 0], [0, 0], [0, 0]])
                
                batch_images.append(img)
                batch_texts.append(texts[:5])  # Берем ровно 5 текстов
                batch_boxes.append(boxes[:5])   # И 5 соответствующих bbox
            
            yield np.array(batch_images), {
                'output_text': np.array(batch_texts),
                'output_box': np.array(batch_boxes)
            }

In [14]:
train_gen = yolo_to_keras_generator(train_dataset, batch_size=16)
val_gen = yolo_to_keras_generator(val_dataset, batch_size=16)
images, labels = next(train_gen)

print("Изображения:", images.shape)
print("Тексты:", labels['output_text'].shape)
print("Боксы:", labels['output_box'].shape)

Изображения: (16, 640, 640, 3)
Тексты: (16, 5)
Боксы: (16, 5, 4, 2)


In [22]:
# Параметры
batch_size = 16
target_size = (640, 640)  # Все изображения будут приведены к этому размеру

In [23]:
# Инициализация пайплайна (детектор + распознаватель)
pipeline = keras_ocr.pipeline.Pipeline()

# Получение модели распознавания текста (можно дообучать)
recognizer = pipeline.recognizer

Looking for /home/lastinm/.keras-ocr/craft_mlt_25k.h5
Looking for /home/lastinm/.keras-ocr/crnn_kurapan.h5


In [24]:
recognizer.model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',  # Для классификации символов
    metrics=['accuracy']
)

In [25]:
history = recognizer.model.fit(
    train_gen,  # Генератор из предыдущего шага
    steps_per_epoch=len(train_dataset) // batch_size,
    validation_data=val_gen,
    validation_steps=len(val_dataset) // batch_size,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint('recognizer_best.h5', save_best_only=True),
        tf.keras.callbacks.EarlyStopping(patience=3),
        tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]
)

Epoch 1/10


ValueError: in user code:

    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/lastinm/PROJECTS/credit_cards_detection/.venv/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.w